In [50]:
import torch
x = torch.tensor([
    [ 0.0470, -0.0431,  0.0421],
    [ 0.0282, -0.0119,  0.0402],
    [0.0181, 0.0000, 0.0000],
    [-0.0017,  0.0001, -0.0004],
    [ 4.1447e-03, -2.7173e-03, -2.3842e-07]
    ])
x.softmax(1)

tensor([[0.3438, 0.3141, 0.3421],
        [0.3364, 0.3232, 0.3404],
        [0.3374, 0.3313, 0.3313],
        [0.3330, 0.3336, 0.3334],
        [0.3346, 0.3323, 0.3332]])

In [81]:
torch.softmax(-x*20, 1)

tensor([[0.1225, 0.7424, 0.1351],
        [0.2490, 0.5552, 0.1958],
        [0.2582, 0.3709, 0.3709],
        [0.3403, 0.3282, 0.3315],
        [0.3093, 0.3548, 0.3360]])

In [71]:
torch.softmax(x.exp()-1, 1)

tensor([[0.3438, 0.3141, 0.3421],
        [0.3364, 0.3230, 0.3406],
        [0.3374, 0.3313, 0.3313],
        [0.3330, 0.3336, 0.3334],
        [0.3346, 0.3323, 0.3332]])

In [70]:
x.exp()-1

tensor([[ 4.8122e-02, -4.2184e-02,  4.2999e-02],
        [ 2.8601e-02, -1.1829e-02,  4.1019e-02],
        [ 1.8265e-02,  0.0000e+00,  0.0000e+00],
        [-1.6986e-03,  1.0002e-04, -3.9995e-04],
        [ 4.1533e-03, -2.7136e-03, -2.3842e-07]])

In [54]:
x.exp()/2

tensor([[0.5241, 0.4789, 0.5215],
        [0.5143, 0.4941, 0.5205],
        [0.5091, 0.5000, 0.5000],
        [0.4992, 0.5001, 0.4998],
        [0.5021, 0.4986, 0.5000]])

In [72]:
x1 = x - x.min(1, keepdim=True)[0] + 1e-3
x1 / x1.sum(1, keepdim=True)

tensor([[0.5109, 0.0056, 0.4835],
        [0.4317, 0.0105, 0.5578],
        [0.9052, 0.0474, 0.0474],
        [0.1639, 0.4590, 0.3770],
        [0.6250, 0.0795, 0.2955]])

# TOXIC dpo

- turn exach pair into ds row
- shuffle

In [89]:
from datasets import load_dataset
dataset = load_dataset('unalignment/toxic-dpo-v0.1') # this should give a a bigger # use map in batched mode to return more rows than it got
def transform(row):
    return {
        "chosen": [{'role':'user', 'content': row['prompt']},{'role':'assistant', 'content': row['chosen']}],
        "rejected": [{'role':'user', 'content': row['prompt']},{'role':'assistant', 'content': row['rejected']}]
    }

dataset1 = dataset.map(transform)
dataset1

DatasetDict({
    train: Dataset({
        features: ['id', 'source', 'prompt', 'rejected', 'chosen'],
        num_rows: 302
    })
})

## Download model

In [84]:
from transformers import AutoTokenizer, AutoModelForCausalLM
model_name = "NousResearch/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model-00001-of-00004.safetensors:  29%|##9       | 1.46G/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

# HelpSteer2

https://github.com/jondurbin/bagel/blob/3c7d2410a5a5ad2fd31b63529ef541135feefce4/bagel/data_sources/helpsteer.py#L4

In [105]:
from datasets import load_dataset
dataset = load_dataset('nvidia/HelpSteer2')
dataset

Generating train split:   0%|          | 0/20324 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1038 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'response', 'helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity'],
        num_rows: 20324
    })
    validation: Dataset({
        features: ['prompt', 'response', 'helpfulness', 'correctness', 'coherence', 'complexity', 'verbosity'],
        num_rows: 1038
    })
})

In [ ]:
import re
from loguru import logger

REFUSALS = [
    re.compile(regexp)
    for regexp in [
        "(against|violate) my programming",
        "openai",
        "language model",
        "large language",
        "as an? (ai|generative language|gpt|bot)",
        "important to note",
        "i do(n't| not) (possess|have|exhibit) (personal|consciousness|subjective)",
        "personal (feelings|thoughts|emotions|desires|experiences|goals|objective|belief)",
        "(can('t| ?not)|w(on't|will not)|unable.?) (\\w+\\s)+(with (that|your)|your \\w+|provide)",
        "my limitations",
        "the limitations of my",
        "my abilities",
        "violates my",
        "i (can('t| ?not)|w(on't|will not)|am (not |un)able.?).{0,30}(you are|you're|your )",
        "please note that",
        "unethical|illegal|dangerous",
        "a text-based",
        "(engag(e|ing)|participat(e|ing)|be involved (in|with)|promot(e|ing)|discuss(ing)?|provid(e|ing))( in)?(\\s*\\w+ that)?(\\s+potentially)? (derogatory|inappropriate|offensive|discriminate|discriminatory|sexist|unacceptable|immoral|unethical|unacceptable|hateful|harmful)",
        "i am commited to",
        "adhere to safety guidelines",
        "maintain user safety",
        "about something else instead",
    ]
]

def has_refusal(response):
    """Check if a response has a refusal - not perfect, but good enough..."""
    for regexp in REFUSALS:
        if m := regexp.search(response, re.I):
            logger.warning(f"Refusal? {m.group()}")
            return True
    return False

.filter(
        lambda item: not has_refusal(item["response"])
    )

In [ ]:
def load_data(known_uids=set([]), **_):
    """HelpSteer dataset for DPO."""
    logger.info("Loading HelpSteer dataset...")
    dataset = load_dataset("nvidia/HelpSteer", split="train").filter(
        lambda item: not has_refusal(item["response"])
    )

    # Use the highest correctness items as the "chosen" responses.
    logger.info("Generating DPO pairs...")
    options = {}
    for item in dataset:
        uid = get_uid(item["prompt"])
        if uid not in options:
            options[uid] = []
        item["score"] = (
            item["helpfulness"]
            + item["correctness"]
            + item["coherence"]
            + item["complexity"]
            + item["verbosity"]
        )
        options[uid].append(item)

    # Select the chosen and rejected items.
    data = []
    for uid, responses in options.items():
        if len(responses) < 2:
            continue
        if uid in known_uids:
            continue
        known_uids.add(uid)
        chosen = None
        chosen_score = 0
        rejected = None
        rejected_score = 0
        for item in responses:
            if item["correctness"] == 4:
                if chosen:
                    if item["score"] > chosen["score"]:
                        rejected = chosen["response"]
                        rejected_score = chosen["score"]
                        chosen = item
                        chosen_score = item["score"]
                        break
                else:
                    chosen = item
                    chosen_score = item["score"]
            else:
                if not rejected:
                    rejected = item["response"]
                    rejected_score = item["score"]
        if chosen and rejected and chosen["response"] != rejected:
            logger.success(f"Found DPO pair: {chosen_score} vs {rejected_score}")
            data.append(
                {
                    "id": uid,
                    "source": "helpsteer",
                    "prompt": item["prompt"],
                    "chosen": chosen["response"],
                    "rejected": rejected,
                    "conversations": None,
                }
            )
    return Dataset.from_list(data)

In [108]:
dataset['train']['prompt']

['c#',
 'c#',
 'bacillus subtilus',
 'bacillus subtilus',
 'Write long detailed essay about ancient type of religion, totemism',
 'Write long detailed essay about ancient type of religion, totemism',
 'Define Signal Discuss its various properties with the help of diagram',
 'Define Signal Discuss its various properties with the help of diagram',
 'how do you prepare potatos for frying',
 'how do you prepare potatos for frying',
 'Please, use the Bloom Taxonomy to develop a professional development plan for learner assessment and evaluation',
 'Please, use the Bloom Taxonomy to develop a professional development plan for learner assessment and evaluation',
 'how can my software company use Microsoft ENTRA to verify the identity of a user before accessing the software?',
 'how can my software company use Microsoft ENTRA to verify the identity of a user before accessing the software?',
 'My team needs to implement Cisco Customer Identity login on our application. Please describe the tasks